# Aula 6 - ALS

In [1]:
!pip install pyspark

In [2]:
!pip install findspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row # formato que ALS trabalha

In [4]:
spark = SparkSession.builder.master('local[*]').getOrCreate()